# Final Capstone Projects

Please refer to the [**Final Capstone Projects**](http://nbviewer.jupyter.org/github/jmportilla/Complete-Python-Bootcamp/tree/master/Final%20Capstone%20Projects/) folder to get all the info on final capstone project ideas and possible solutions!

# <center>Sort Excel/CSV File Utility </center>

<h2>Modules Import</h2>

In [1]:
#import openpyxl
#import csv
#import os

import pandas as pd
import tkinter as tk

from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import askquestion
from tkinter.messagebox import askokcancel
from tkinter.messagebox import showerror
from tkinter import ttk
import getpass

#from tkinter import *
from functools import partial

#Constants Variables
supported_formats = ["xlsx","xls","xlsm","xlsb","csv"]

 <h2>Functions</h2>

In [2]:
def file_name_spliter(f_full_path):
    """Split the file name and his extension
    
    Parameters
    --------------------------------------
    f_full_path: the full path to the file
    """
    f_file_name = f_full_path.split("/")[-1].split(".")[0]
    f_file_extension =  f_full_path.split("/")[-1].split(".")[-1] 
    
    return f_file_name, f_file_extension

In [3]:
def right_format(f_file_name,f_file_extension):
    """Split the file name and his extension
    
    Parameters
    --------------------------------------
    f_file_name: the name of the file
    f_file_extension: the exension of the file
    """
    if f_file_extension not in supported_formats:
        msg = f"The file \"{f_file_name}\" is on \"{f_file_extension}\" format and is not supported by this program \n" \
              +"\nYou can see below the list of the supported format:\n\n" \
              + "\n".join(supported_formats)
        showerror(title='Error of file format', message=msg)
        return False
    else:
        return True

In [4]:
def get_data(f_path,f_extension):
    """Get Data from xlsx or csv File
    
    Parameters
    --------------------------------------
    f_path: the full path to the file
    f_extension: the extension of the file
    """
    if f_extension == "csv":
        data = pd.read_csv(f_path,delimiter=';')
    else:
        data = pd.read_excel(f_path)
    return data

<h2>Classes</h2>

In [7]:
class Application(tk.Tk):
    
    def __init__(self):
        tk.Tk.__init__(self)
        self.geometry("800x500")
        self.create_widgets()
       
    def create_widgets(self):
        self.frame1 = tk.LabelFrame(self, text="Excel Data")
        self.frame2 = tk.LabelFrame(self, text="Open File")
        self.bouton1 = tk.Button(self.frame2, text="Load a file", command=self.load_data)
        self.bouton2 = tk.Button(self.frame2, text="Sort the Data", command="")
        self.bouton3 = tk.Button(self.frame2, text="Quitter", command=self.destroy)
        
        self.comboBox = ttk.Combobox(self, values=[])
        self.labelTop = tk.Label(self,text = "Choose the field whith you want to sort the file")
        self.tv1 = ttk.Treeview(self.frame1)
        
        #adding scrollbar hoziontal and vertical
        self.scrollY = tk.Scrollbar(self, orient=tk.VERTICAL, command=self.tv1.yview)
        self.scrollX = tk.Scrollbar(self, orient=tk.HORIZONTAL, command=self.tv1.xview)
        
        self.tv1.place(relheight=1, relwidth=1) 
        
        self.frame1.place(height=300, width=640, rely=0.05, relx=0.1)
        self.scrollY.place(relx=0.90,rely=0.05,height=300)
        self.scrollX.place(relx=0.1,rely=0.65,width=640)
        
        print(self.frame1.winfo_width())
        print(self.frame1.winfo_width())
        
        self.labelTop.place(relx=0.13,rely=0.7)
        self.comboBox.place(width=320,rely=0.75,relx=0.1)
        self.frame2.place(height=50, width=640, rely=0.83, relx=0.1)
        self.bouton1.place(relx=0.03)
        self.bouton2.place(relx=0.4)
        self.bouton3.place(relx=0.85)
        
    #Choose a directory and a file
    def load_data(self):
       
        tk.Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
        file_path = askopenfilename() # show an "Open" dialog box and return the path to the selected file

        if file_path != "":

            #Get the extenion of the file
            file_name, file_extension = file_name_spliter (file_path)

            #Getting the data from the file
            if right_format(file_name, file_extension):
                data = get_data(file_path,file_extension)
                
                #adding columns
                self.comboBox['values']= list(data.columns)
                self.comboBox.current(0)
                self.tv1["column"]= list(data.columns)
                self.tv1["show"] = "headings"
                
                for column in self.tv1["columns"]:
                    self.tv1.heading(column, text=column) # let the column heading = column name
                
                #adding rows of data
                df_rows = data.to_numpy().tolist() # turns the dataframe into a list of lists
                for row in df_rows:
                    self.tv1.insert("", "end", values=row)  
                    
                #self.tv1["displaycolumns"]= "#all"
            
                

if __name__ == "__main__":
    app = Application()
    app.title("Sort Excel/CSV File Utility")
    app.mainloop()

1
1


In [ ]:
tk.LabelFrame.winfo_width

In [ ]:
#Choose the header to sort the data with 
#app =Tk.Tk()
#data.sort_values(by='First Name',inplace=True)
#data.to_excel("dataframe_to_xlsx.xlsx", index=False)
list(data.columns)

In [50]:
app = Tk() 
app.geometry('500x500')

labelTop = tk.Label(app,text = "Choose the field whith you want to sort the file")
labelTop.grid(column=0, row=0)

labelHeaders = tk.Label(app,text = list(data.columns))
labelHeaders.grid(column=0, row=1)

comboExample = ttk.Combobox(app, values=list(data.columns))
comboExample.grid(column=0, row=2)
comboExample.current(0)

frame1 = tk.LabelFrame(app, text="Excel Data")
frame1.place(rely=0.65, relheight=1,relwidth=1)

app.mainloop()

KeyboardInterrupt: 

In [ ]:
ws['A1'] = 42

In [ ]:
ws.rows

In [ ]:
# Rows can also be appended
ws.append([1, 2, 3])

# Python types will automatically be converted
import datetime
ws['A2'] = datetime.datetime.now()

# Save the file
wb.save("sample.xlsx")